# Session #9 Homework

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

## Question 1

Convert this model from Keras to TF-Lite format.  
What's the size of the converted model?

In [2]:
model = keras.models.load_model("dogs_cats_10_0.687.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("dogs_cats_10_0.687.tflite", "wb") as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\mdrak\AppData\Local\Temp\tmprvau9lhj\assets


In [3]:
ls

 Volume in drive D is Files
 Volume Serial Number is 5850-A0F0

 Directory of D:\Development\mlcamp\09-serverless

29 Nov 2021  00:52    <DIR>          .
28 Nov 2021  15:42    <DIR>          ..
28 Nov 2021  18:58    <DIR>          .ipynb_checkpoints
28 Nov 2021  15:43        89.758.304 dogs_cats_10_0.687.h5
29 Nov 2021  00:52        44.865.956 dogs_cats_10_0.687.tflite
29 Nov 2021  00:52            10.521 homework.ipynb
28 Nov 2021  15:43             4.559 homework.md
28 Nov 2021  23:51            99.389 Pug_600.jpg
               5 File(s)    134.738.729 bytes
               3 Dir(s)  30.448.893.952 bytes free


The size of the converted model is approximately **43MB**.

## Question 2

To be able to use this model, we need to know the index of the input and the index of the output.  
What's the output index for this model?

In [4]:
interpreter = tflite.Interpreter(model_path="dogs_cats_10_0.687.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(f"Input index: {input_index}, Output index: {output_index}")

Input index: 0, Output index: 13


Output index is **13**.

## Preparing the Image

In [5]:
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

The target size of the image should be **(150,150)**.

In [6]:
url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"
downloaded_image = download_image(url)
prepared_image = prepare_image(downloaded_image, (150,150))

## Question 3

Now we need to turn the image into an numpy array and pre-process it.  
After the pre-processing, what's the value in the first pixel, the R channel?

In [30]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [31]:
x = np.array(prepared_image, dtype="float32")
X = np.array([x])
X = preprocess_input(X)

In [32]:
X

array([[[[ 0.41176474,  0.54509807,  0.49803925],
         [ 0.38823533,  0.52156866,  0.45882356],
         [ 0.26274514,  0.39607847,  0.33333337],
         ...,
         [ 0.5372549 ,  0.69411767,  0.6784314 ],
         [ 0.36470592,  0.4901961 ,  0.4901961 ],
         [ 0.36470592,  0.5294118 ,  0.5372549 ]],

        [[ 0.05098045,  0.18431377,  0.13725495],
         [ 0.47450984,  0.60784316,  0.56078434],
         [ 0.21568632,  0.34901965,  0.28627455],
         ...,
         [ 0.58431375,  0.73333335,  0.7176471 ],
         [ 0.26274514,  0.4039216 ,  0.2941177 ],
         [ 0.5764706 ,  0.75686276,  0.70980394]],

        [[ 0.37254906,  0.49803925,  0.47450984],
         [ 0.23921573,  0.37254906,  0.32549024],
         [ 0.2941177 ,  0.427451  ,  0.36470592],
         ...,
         [ 0.20000005,  0.3176471 ,  0.2941177 ],
         [ 0.254902  ,  0.38823533,  0.30980396],
         [ 0.41176474,  0.58431375,  0.56078434]],

        ...,

        [[ 0.16078436,  0.23921573,  0

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [10]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.97000027]], dtype=float32)

Prediction of the model is **0.97**.